In [1]:
!nvidia-smi

Thu Aug 29 06:32:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [3]:
!pip install ultralytics==8.0.196

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 38.4 MB/s eta 0:00:00


In [4]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 32.8/201.2 GB disk)


In [5]:
from ultralytics import YOLO

from IPython.display import display, Image

## Inference with Pre-trained COCO Model

### 💻 CLI

`yolo mode=predict` runs YOLOv8 inference on a variety of sources, downloading models automatically from the latest YOLOv8 release, and saving results to `runs/predict`.

In [6]:
!yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source='/content/wood.jpeg' save=True

100% 6.23M/6.23M [00:00<00:00, 318MB/s]
/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:567: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(

### 🐍 Python SDK

The simplest way of simply using YOLOv8 directly in a Python environment.

In [12]:
model = YOLO('/content/yolov8n.pt')
results = model.predict(source='/content/wood.jpeg', save=True,save_txt=True)


WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/1 /content/wood.jpeg: 448x640 1 donut, 39.9ms
Speed: 11.2ms preprocess, 39.9ms inference, 751.5ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict
1 label saved to runs/detect/predict/labels


In [13]:
results[0].boxes.xyxy

tensor([[261.4549,  95.2741, 854.3775, 619.6642]], device='cuda:0')

In [14]:
results[0].boxes.conf

tensor([0.6461], device='cuda:0')

In [15]:
results[0].boxes.cls

tensor([54.], device='cuda:0')

In [16]:
!mkdir /content/datasets
%cd /content/datasets


mkdir: cannot create directory ‘/content/datasets’: File exists
/content/datasets


In [17]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="YgB7C0B6gi2t4gCZsfzG")
project = rf.workspace("jjj-5g4ds").project("numberwood")
version = project.version(2)
dataset = version.download("yolov8-obb")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.8
    Uninstalling idna-3.8:
      Successfully uninstalled idna-3.8
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to numberwood-2 in yolov8-obb:: 100%|██████████| 668/668 [00:00<00:00, 5589.56it/s]


In [18]:
%cd /content

/content


## Custom Training

In [19]:
ls /content/datasets/numberwood-2/valid/


images/  labels/


In [29]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/datasets/numberwood-2/data.yaml epochs=50 imgsz=800 plots=True

/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:567: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


In [33]:
!yolo task=detect mode=predict model=/content/runs/detect/train/weights/best.pt conf=0.25 source=/content/wood1.jpg save=True

/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:567: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


In [34]:
!yolo task=detect mode=predict model=/content/runs/detect/train/weights/best.pt conf=0.25 source=/content/wood2.jpg save=True

/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:567: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


In [35]:
!yolo task=detect mode=predict model=/content/runs/detect/train/weights/best.pt conf=0.25 source=/content/wood3.jpg save=True

/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:567: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


In [ ]:
pip install google-colab


  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [25]:
from google.colab import drive

# เชื่อมต่อ Google Drive
drive.mount('/content/drive')

# ตอนนี้คุณสามารถเข้าถึง Google Drive ของคุณได้จาก '/content/drive/My Drive/'


Mounted at /content/drive


In [ ]:
import shutil

# เส้นทางของโฟลเดอร์ที่ได้จากการเทรน
source_folder_train2 = '/content/runs/detect/train2'
source_folder_train3 = '/content/runs/detect/train3'

# เส้นทางของไดรฟ์ปลายทางที่ต้องการบันทึก
destination_drive = '/content/drive/MyDrive/Yolyo/wight'


# คัดลอกโฟลเดอร์ train2 ไปยังไดรฟ์ปลายทาง
shutil.copytree(source_folder_train2, destination_drive + 'train2')

# คัดลอกโฟลเดอร์ train3 ไปยังไดรฟ์ปลายทาง
shutil.copytree(source_folder_train3, destination_drive + 'train3')


FileExistsError: [Errno 17] File exists: '/content/drive/MyDrive/Yolyo/wighttrain2'

In [ ]:
!pip install torch torchvision torchaudio
import torch

# เส้นทางไปยังโฟลเดอร์ใน Google Drive
save_drive_path = '/content/drive/MyDrive/Yolyo/wight'

# บันทึกโมเดลไปยัง Google Drive
# Use the existing 'model' variable to save your trained YOLO model
torch.save(model, save_drive_path + 'model_train.pth')

In [ ]:
pip install torch flask

In [ ]:
!pip install ultralytics
import torch
import torch.nn as nn
from flask import Flask, request, jsonify
from PIL import Image
import io
import torchvision.transforms as transforms
from ultralytics import YOLO

# Define your model class (if you have a custom model architecture)
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # Define layers here
        self.fc = nn.Linear(128, 3)  # Example layer

    def forward(self, x):
        # Define forward pass
        return self.fc(x)

# Initialize the Flask app
app = Flask(__name__)

# Load the model
def load_model(model_path):
    model = MyModel()  # Replace with the actual model class if different
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.eval()  # Set the model to evaluation mode
    return model

model = load_model('/content/drive/MyDrive/Yolyo/wight/Yolyotrain3/weights/best.pt')

# Define image preprocessing function
def transform_image(image_bytes):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Replace with your normalization
    ])
    image = Image.open(io.BytesIO(image_bytes))
    return transform(image).unsqueeze(0)

# Define prediction function
def get_prediction(image_bytes):
    tensor = transform_image(image_bytes)
    outputs = model(tensor)
    _, predicted = torch.max(outputs.data, 1)
    return predicted.item()

# Set up a route for prediction
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'}), 400

    file = request.files['file']
    img_bytes = file.read()
    prediction = get_prediction(img_bytes)
    return jsonify({'prediction': prediction})

if __name__ == '__main__':
    app.run(debug=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 15.6 MB/s eta 0:00:00


RuntimeError: Error(s) in loading state_dict for MyModel:
	Missing key(s) in state_dict: "fc.weight", "fc.bias". 
	Unexpected key(s) in state_dict: "epoch", "best_fitness", "model", "ema", "updates", "optimizer", "train_args", "train_metrics", "train_results", "date", "version". 